# <center> Fetching data from Riot API server </center>

### Pulls recent match details from players listed in Tier/Division ranking
#### Read more about the LOL API [here](https://developer.riotgames.com/docs/portal)!

#### The notebook goes through the following steps:

1. Pulling summoner names from leagues and saving table to data folder
2. Pulling accountid for summoners names to be able to get match details
3. Pulling match history for each player
4. Pulling game details for each matchID

In [1]:
from JinxData import LOL_API_Connection as LOL
import pandas as pd
from time import sleep
from time import time
from datetime import datetime
from ratelimit import limits, sleep_and_retry
import json
import os

In [2]:
ROOT_DIR = '.'
DATA_DIR = 'data'

### Insert LOL API key and Region here:

In [3]:
API_key = 'RGAPI-e502b975-1fbe-4499-9b60-c67fb91a9932'
region = 'eun1'

In [4]:
# Setting up connection
api = LOL(API_key, region)

**The rate limit for a personal keys is limited:**
- 20 requests every 1 second
- 100 requests every 2 minutes

**Below you can adjust the number of *calls* within a specific *period* (*s*):**

In [5]:
calls = 100
period = 123
@sleep_and_retry
@limits(calls=calls, period=period)
def get_response(conn_type, value_list, i, **filters):
    return conn_type(value_list[i], **filters)

def get_records(conn_type, value_list, verbose = 100, sleeptime = 1, **filters):
    '''Pull records from the list with a specific data collector.
    Returns a list with the requested records as dicts. Accepts filters as **kwargs'''
    newdict = {} 
    errors = []
    i = 0
    while i < len(value_list):

        newdict[value_list[i]] = get_response(conn_type, value_list, i, **filters)
                
        if 'status' in newdict[value_list[i]]:                                              # error handling
            status = newdict[value_list[i]]['status']
            print('{} Error: {}'.format(datetime.now().strftime('%H:%M:%S'), str(status) ) )
            errors.append([time(), status['status_code']] )
            maxerror = 25
            del newdict[value_list[i]]
            if len(errors) >= maxerror and errors[-1][0] - errors[-maxerror][0] < 2*60:     # break loop if too many errors within an interval
                print(errors)
                print('Session interupted due to excessive errors')
                break
            elif errors[-1][1] == 404:
                i +=1
            i -= 1
            sleep(2)
        else:
            sleep(sleeptime)
            #counter = len(value_list) - i -1
            if verbose != 0 and i % verbose == 0:
                print('{0}: Loaded: {1} {2:.0f}% | Time remaining: {3:0.1f} min'.format(datetime.now().strftime('%H:%M:%S'), i, int((i+1)/len(value_list)*100), (len(value_list) - i -1)*1.2/60 ) )
        i += 1
    print('---FINISHED---')
    return newdict

--------------------------
### 1. Pulling summoner names from leagues and saving table to data folder

In [6]:
divisions = ['I','II','III','IV']
tiers = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON']

Number of request: 4 (divisions) x 6 (tiers) = 24

In [7]:
league_names = {}
for tier in tiers:
    for div in divisions:
        league_names[tier + '-' + div] = api.GetSummonerNames(division = div, tier = tier)

In [8]:
summoner_list = pd.DataFrame()
for key in league_names:
    summoner_list = summoner_list.append( pd.DataFrame( league_names[key]), ignore_index= True )

In [9]:
summoner_list.head(3)

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries
0,9072a5a9-717a-49d9-836f-60f491db10dc,RANKED_SOLO_5x5,DIAMOND,I,qXPT97QKWFI1jvEkENZcPzeb-jBNWIETJISe8opfPW1k,Just Milánek,67,23,22,False,False,False,False,NaN
1,e1d88385-83f4-4293-82db-c3b40845f07a,RANKED_SOLO_5x5,DIAMOND,I,hpvIXwvNeJLyyajKKr4BrG4dddpBUqpy6C4kMcpd0KtSwVA,Vamkiller,26,22,18,False,False,False,False,NaN
2,5d15e213-8623-42b9-a0ac-54311251fef7,RANKED_SOLO_5x5,DIAMOND,I,qPdFedw3wxod2C85g58Y6wxJ61hJRCrgH1VfQzzF7vEw73I,Anarxia,0,155,141,False,False,False,False,NaN


In [10]:
#Saving to csv
summoner_list.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_list.csv'))

-----------------------
## 2. Pulling accountid for summoners names to be able to get match details

In [11]:
#Loading in saved csv file
summoner_list = pd.read_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_list.csv'))
summonerName = list(summoner_list.summonerName)

#### Requesting summonerIds name by name. This may take up to 2 hours beacuase of the rate limitations.

In [12]:
sumIDs = get_records(api.GetSummonerID, summonerName[0:10], verbose = 1)

14:31:52: Loaded: 0 10% | Time remaining: 0.2 min
14:31:53: Loaded: 1 20% | Time remaining: 0.2 min
14:31:55: Loaded: 2 30% | Time remaining: 0.1 min
14:31:56: Loaded: 3 40% | Time remaining: 0.1 min
14:31:57: Loaded: 4 50% | Time remaining: 0.1 min
14:31:59: Loaded: 5 60% | Time remaining: 0.1 min
14:32:00: Loaded: 6 70% | Time remaining: 0.1 min
14:32:01: Loaded: 7 80% | Time remaining: 0.0 min
14:32:03: Loaded: 8 90% | Time remaining: 0.0 min
14:32:04: Loaded: 9 100% | Time remaining: 0.0 min
---FINISHED---


In [13]:
id_list = pd.DataFrame( [sumIDs[key] for key in sumIDs.keys()] )
id_list.head()

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,qXPT97QKWFI1jvEkENZcPzeb-jBNWIETJISe8opfPW1k,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU,r-fCz1Lws921xxhK6TdrpnvdhASTJ93YqVV41rQaZ1XF7Q...,Just Milánek,524,1583102552000,192
1,hpvIXwvNeJLyyajKKr4BrG4dddpBUqpy6C4kMcpd0KtSwVA,gVDSgGNiPyajVqzsqGbRCQZe8xkYgd0vCB6nPzgPRGYrdA,Nv-SyN3E4Fa1TB03McOQkz92HgKVZJPCQqkkA4Uz1RgyhO...,Vamkiller,519,1583283763000,193
2,qPdFedw3wxod2C85g58Y6wxJ61hJRCrgH1VfQzzF7vEw73I,8mLIYnaVybot7GScJ0US1DcK0s4swTrnQXWMkAaPEFJ2fA,0t01JIaZ-x4jZmJYatDXSxnjS0-Nrxpnso8-WVNW8iRLSK...,Anarxia,522,1583249933000,126
3,NyF7brLZQUxCqBte3VQhEWPA0LshKJDZ8tgYkLo_paPO3Cw,42HsbZLgKUcnvQYPqEmj7yWfGk6Ccic2PNYvbjrKX3os9w,Rqa-RES4KBre_b3JGqxQ7ICSqdIcGuQA8d-WmOa8QZc_js...,PoZeRaKoC LoStRe,14,1583097414000,98
4,XLnTOSYk15rImfcdBgmF68tw2-JkuUOAVrKZBPY02PTBKRg,y5BE9bBJURje7QGpxx12GOp9_7PWGg6Wtw1zoWpzDOMG4A,veW6sstFZhOQgCsPEhA3H_W93Ju-1bCLEq-WHVJN3jDjQQ...,Prwktolog0s,585,1583285564000,105


In [14]:
#Saving to csv
id_list.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_id_details.csv'))

----------------------
## 3. Pulling match history for each player

In [15]:
#Loading in saved csv file
id_list = pd.read_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_id_details.csv'))
accountId = list(id_list['accountId'])

**This may take several hours**

In [16]:
match_hist = get_records(api.GetMatchHist_full, accountId[0:10], verbose=1, sleeptime=0) 

14:32:05: Loaded: 0 10% | Time remaining: 0.2 min
14:32:05: Loaded: 1 20% | Time remaining: 0.2 min
14:32:06: Loaded: 2 30% | Time remaining: 0.1 min
14:32:06: Loaded: 3 40% | Time remaining: 0.1 min
14:32:07: Loaded: 4 50% | Time remaining: 0.1 min
14:32:07: Loaded: 5 60% | Time remaining: 0.1 min
14:32:07: Loaded: 6 70% | Time remaining: 0.1 min
14:32:08: Loaded: 7 80% | Time remaining: 0.0 min
14:32:08: Loaded: 8 90% | Time remaining: 0.0 min
14:32:09: Loaded: 9 100% | Time remaining: 0.0 min
---FINISHED---


In [17]:
match_list = pd.DataFrame()
for key, value in match_hist.items():
    id_hist = pd.DataFrame(value)
    id_hist['accountId'] = key
    match_list = match_list.append(id_hist)
match_list.reset_index(inplace = True)
match_list.head()

,index,platformId,gameId,champion,queue,season,timestamp,role,lane,accountId
0,0,EUN1,2394936195,105,420,13,1583100333332,DUO,MID,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU
1,1,EUN1,2394931561,1,700,13,1583098541514,SOLO,MID,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU
2,2,EUN1,2394894769,101,700,13,1583096058349,SOLO,MID,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU
3,3,EUN1,2394882761,1,700,13,1583092551254,SOLO,MID,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU
4,4,EUN1,2394776541,69,420,13,1583087862983,SOLO,MID,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU


In [18]:
#Saving to csv
match_list.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'match_history.csv'))

In [19]:
match_list.shape

(1000, 10)

--------------------
## 4. Pulling game details for each matchID
This part creates a new folder in data dir then saves each player's match detail into a separate json file 

In [20]:
#Loading in saved csv file
match_list = pd.read_csv(os.path.join(ROOT_DIR, DATA_DIR, 'match_history.csv'))

In [21]:
gameids = match_list[['gameId', 'accountId']]
gameids.head(2)

,gameId,accountId
0,2394936195,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU
1,2394931561,sP1WpJtn8ddDSp6eDbVcYQXOVuqSKGt1bOf0aRGZJHU


In [22]:
path = os.path.join(ROOT_DIR, DATA_DIR, 'match_details_by_summoners')
os.makedirs(path, exist_ok=True)

In [23]:
def save_file(path, name, data):
    filename = os.path.join(path, name + '.json')
    jsonfile = json.dumps(data)
    file = open(filename, 'w')
    file.write(jsonfile)
    file.close()

In [24]:
# Removing items that were already downloaded (already in the downloaded directory)
dir_content = [s.replace('.json','') for s in os.listdir(path)]

full_accountid_list = list(gameids['accountId'].unique()[0:4])  # Now only pulls 5 players match details

accountid_list = [s for s in full_accountid_list if s not in dir_content]

In [25]:
for id in accountid_list:
    games = list(gameids[ gameids.accountId == id]['gameId'])
    
    # Get all game details in the games list
    game_detail_dict = get_records(api.GetMatchDetails, games, verbose=8 , sleeptime=0.3)
     
    save_file(path, id, game_detail_dict)
    print('{}% of files saved!'.format( int((accountid_list.index(id)+1)/len(accountid_list)*100)) )

14:34:31: Loaded: 0 1% | Time remaining: 2.0 min
14:34:38: Loaded: 8 9% | Time remaining: 1.8 min
14:34:44: Loaded: 16 17% | Time remaining: 1.7 min
14:34:50: Loaded: 24 25% | Time remaining: 1.5 min
14:34:56: Loaded: 32 33% | Time remaining: 1.3 min
14:35:03: Loaded: 40 41% | Time remaining: 1.2 min
14:35:09: Loaded: 48 49% | Time remaining: 1.0 min
14:35:15: Loaded: 56 56% | Time remaining: 0.9 min
14:35:22: Loaded: 64 65% | Time remaining: 0.7 min
14:35:28: Loaded: 72 73% | Time remaining: 0.5 min
14:35:35: Loaded: 80 81% | Time remaining: 0.4 min
14:35:41: Loaded: 88 89% | Time remaining: 0.2 min
14:35:48: Loaded: 96 97% | Time remaining: 0.1 min
---FINISHED---
25% of files saved!
14:36:34: Loaded: 0 1% | Time remaining: 2.0 min
14:36:41: Loaded: 8 9% | Time remaining: 1.8 min
14:36:47: Loaded: 16 17% | Time remaining: 1.7 min
14:36:53: Loaded: 24 25% | Time remaining: 1.5 min
14:37:00: Loaded: 32 33% | Time remaining: 1.3 min
14:37:06: Loaded: 40 41% | Time remaining: 1.2 min
14:3

----------------------
# Merging all match detail files to masterdata

In [26]:
from ProgressBar import update_progress

In [27]:
path = os.path.join(ROOT_DIR, DATA_DIR, 'match_details_by_summoners')

In [28]:
def game_participants_to_df(data, gameid):
    '''Merging player details and match details in one file'''
    players_in_game = pd.DataFrame()
    n_of_participants = len(data[gameid]['participants'])
    for i in range(n_of_participants): # going through the participants and adding other info 
        # 1. Creating participant row with partIdentities
        part_id = data[gameid]['participantIdentities'][i]['participantId']
        participant_row = pd.DataFrame(data[gameid]['participantIdentities'][i]['player'], index=[part_id])

        # 2. Adding games stasts to the row
        stats = pd.DataFrame(data[gameid]['participants'][i]['stats'], index = [part_id] ).rename(columns= {'inhibitorKills': 'inhibitorKills_participant'})
        lane = pd.DataFrame(data[gameid]['participants'][i]['timeline'], index = [part_id] )[['role', 'lane']]
        stats = stats.join(lane)
        newkeys = [key for key in data[gameid]['participants'][i].keys() if key not in ['stats', 'timeline', 'participantId'] ]
        for key in newkeys:
            stats[key] = data[gameid]['participants'][i][key]
        participant_row = participant_row.join(stats)

        # 3. Adding teams stats
        teamstat = pd.DataFrame(data[gameid]['teams']).drop('win', axis = 1).set_index('teamId')
        participant_row = participant_row.merge(teamstat, on = 'teamId')

        # 4. Add row to the total list
        players_in_game = players_in_game.append( participant_row )

        # 5. Adding game desc details to the game dataframe
        newkeys = [key for key in data[gameid].keys() if key not in ['participants', 'participantIdentities', 'teams'] ]
        for key in newkeys:
            players_in_game[key] = data[gameid][key]
    players_in_game.reset_index(drop = True, inplace = True)
    return players_in_game

In [29]:
# Loops through the match_details_by_summoners dir files and 

FolderItems = os.listdir(path)
games = pd.DataFrame()
for file in FolderItems:
    json_file = open(os.path.join(path, file))
    data = json.load(json_file)
    for gameid in data:
        games = games.append(game_participants_to_df(data, gameid))
        games.reset_index(drop = True)
    update_progress((FolderItems.index(file)+1)/len(FolderItems))

Progress: [####################] 100.0%


In [37]:
games.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'match_detail_master.csv') )

In [35]:
games.shape

(4000, 141)